# Кластеризация

Обучение без учителя (unsupervised learning, неконтролируемое обучение) – класс методов машинного обучения для поиска шаблонов в наборе данных. Данные, получаемые на вход таких алгоритмов, обычно не размечены, то есть передаются только входные переменные X без соответствующих меток y. Если в контролируемом обучении (обучении с учителем, supervised learning) система пытается извлечь уроки из предыдущих примеров, то в обучении без учителя система старается самостоятельно найти шаблоны непосредственно из приведенного примера.

Методы кластеризации данных являются одним из наиболее популярных семейств машинного обучения без учителя. Рассмотрим некоторые из них подробнее. Начнем с иерархиеской кластеризации.

## Иерархическая кластеризация 

Иерархическая кластеризация (также графовые алгоритмы кластеризации и иерархический кластерный анализ) — совокупность алгоритмов упорядочивания данных, направленных на создание иерархии (дерева) вложенных кластеров. Выделяют два класса методов иерархической кластеризации:

- Агломеративные методы (англ. agglomerative): новые кластеры создаются путем объединения более мелких кластеров и, таким образом, дерево создается от листьев к стволу;
- Дивизивные или дивизионные методы (англ. divisive): новые кластеры создаются путем деления более крупных кластеров на более мелкие и, таким образом, дерево создается от ствола к листьям.

Алгоритмы иерархической кластеризации предполагают, что анализируемое множество объектов характеризуется определённой степенью связности. По количеству признаков иногда выделяют монотетические и политетические методы классификации. Как и большинство визуальных способов представления зависимостей графы быстро теряют наглядность при увеличении числа кластеров.

Выполним построение дендрограммы методом агломеративной кластеризации.

```python
X = df[col] #какие данные возьмем для кластеризации
#Желательно брать данные после нормализации данных

from scipy.cluster.hierarchy import dendrogram
from sklearn.cluster import AgglomerativeClustering

def plot_dendrogram(model, **kwargs):
    # создадим матрицу связей для построения дендрограммы

    counts = np.zeros(model.children_.shape[0])
    n_samples = len(model.labels_)
    for i, merge in enumerate(model.children_):
        current_count = 0
        for child_idx in merge:
            if child_idx < n_samples:
                current_count += 1  # leaf node
            else:
                current_count += counts[child_idx - n_samples]
        counts[i] = current_count

    linkage_matrix = np.column_stack([model.children_, model.distances_,
                                      counts]).astype(float)

    # Передаем данные для построения дендрограммы
    dendrogram(linkage_matrix, **kwargs)

# устанавливаем distance_threshold=0 чтиобы гарантированно посчитать полное дерево
model = AgglomerativeClustering(distance_threshold=0, n_clusters=None)

model = model.fit(X)
plt.figure(figsize=(15,5)) #размер фигуры
plt.title('Hierarchical Clustering Dendrogram')
# можно установить количество уровней дендрограммы, параметр р
plot_dendrogram(model, truncate_mode='level', p=3)
plt.xlabel("Количество точек в узле (или индекс точки, если нет скобок).")
plt.show()
```
Данная модель наглядна, но на практике чаще используют другой подход.

```python
from scipy.cluster.hierarchy import linkage, fcluster, dendrogram

Z = linkage(X, method='ward') #другие методы {“ward”, “complete”, “average”, “single”}, default=”ward”
plt.figure(figsize=(15,7))
dendrogram(Z, truncate_mode='level')
plt.show()
```

Нам нужна матрица linkage (связей).

```python
# максимизируем количество кластеров (параметр задаем)
max_clusters=fcluster(Z, 10, criterion='maxclust')
max_clusters[:10]
```

```python
#используем в качестве критерия расстояние
d_clusters=fcluster(Z,  t=50000, criterion='distance')
d_clusters[:10]
```

Не всегда просто интерпретировать полученные кластеры.

```python
df_result=df[col].copy()
df_result['max_clusters']=max_clusters
df_result['distance']=d_clusters
df_result.sample(10)
```

Выполним группировку по кластерам.
```python
df_analize=df_result.groupby('max_clusters')[col].mean()
df_analize['Count']=df_result.groupby('max_clusters')['max_clusters'].count()
df_analize
```

Также полезно визуализировать результаты.

```python
import seaborn as sns
import matplotlib.pyplot as plt
 
x=col[0] #Изменяйте столбцы 
y=col[2]
print(x,y)
sns.lmplot( x=x, y=y, data=df_result, fit_reg=False, hue='max_clusters', legend=False)
plt.legend(loc='lower right')
plt.show()
```

Лучше работает при небольшом наборе кластеров.

```python
g = sns.lmplot(x=x, y=y, hue="max_clusters", col="max_clusters",
               data=df_result, height=6, aspect=.4, x_jitter=.1)
```

Есть вариант в две колонки

```python
g = sns.lmplot(x=x, y=y, hue="max_clusters", col="max_clusters",
               data=df_result, col_wrap=2, height=3)
```

In [ ]:
#Желательно брать данные после нормализации данных
min_max_scaler = preprocessing.MinMaxScaler()
X = min_max_scaler.fit_transform(df[col])

# Снижение размерности<a name="i8"></a>

Одной из самых очевидных задач, которые возникают в отсутствие явной разметки, является задача снижения размерности данных. С одной стороны её можно рассматривать как помощь в визуализации данных, c другой стороны подобное снижение размерности может убрать лишние сильно скоррелированные признаки у наблюдений и подготовить данные для дальнейшей обработки в режиме обучения с учителем, например сделать входные данные более "перевариваемыми" для деревьев решений.

Объемы и сложность данных постоянно растут. В результате, существенно увеличивается и их размерность. Для компьютеров это не проблема — в отличие от людей: мы ограничены всего тремя измерениями.

Структура, скрытая в данных, может быть восстановлена только с помощью специальных математических методов. К ним относится подраздел машинного обучения без учителя под названием множественное обучение (manifold learning) или нелинейное уменьшение размерности (nonlinear dimensionality reduction).

## PCA

PCA помогает выразить несколько признаков через один, что позволяет работать с более простой моделью.

PCA аппроксимирует n-размерное облако наблюдений до эллипсоида (тоже n-мерного), полуоси которого и будут являться будущими главными компонентами. И при проекции на такие оси (снижении размерности) сохраняется наибольшее количество информации.

Вычисление главных компонент сводится к вычислению собственных векторов и собственных значений ковариационной матрицы исходных данных или к сингулярному разложению матрицы данных.

```python
from sklearn import decomposition

pca = decomposition.PCA(n_components=2)
X_pca = pca.fit_transform(df[col])

x_axis = X_pca[:, 0]
y_axis = X_pca[:, 1]

plt.scatter(x_axis, y_axis, c=df_result.max_clusters)
plt.show()
```

Можно посмотреть, как образованы главные компоненты.

```python
plt.matshow(pca.components_, cmap='viridis')
plt.yticks([0, 1], ["Первая компонента", "Вторая компонента"])
plt.colorbar()
plt.xticks(range(col), col, rotation=60, ha='left')
plt.xlabel("Характеристика")
plt.ylabel("Главные компоненты")
```

## t-SNE<a name="i9"></a>

t-SNE (t-distributed stochastic neighbor embedding) — техника нелинейного снижения размерности и визуализации многомерных переменных. Этот алгоритм может свернуть сотни измерений к меньшему количеству, сохраняя при этом важные отношения между данными: чем ближе объекты располагаются в исходном пространстве, тем меньше расстояние между этими объектами в пространстве сокращенной размерности. t-SNE неплохо работает на маленьких и средних реальных наборах данных и не требует большого количества настроек гиперпараметров.

Алгоритм t-SNE, который также относят к методам множественного обучения признаков, был опубликован в 2008 году голландским исследователем Лоуренсом ван дер Маатеном (сейчас работает в Facebook AI Research) и Джеффри Хинтоном. Классический SNE был предложен Хинтоном и Ровейсом в 2002. В статье 2008 года описывается несколько «трюков», которые позволили упростить процесс поиска глобальных минимумов, и повысить качество визуализации.

```python
from sklearn.manifold import TSNE

# Определяем модель и скорость обучения
model = TSNE(learning_rate=100)

# Обучаем модель
transformed = model.fit_transform(rob)

x_axis = transformed[:, 0]
y_axis = transformed[:, 1]

plt.scatter(x_axis, y_axis, c=df_result.max_clusters)
plt.show()
```